In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
from typing import Tuple

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# main flow
start_time = dt.datetime.now()
print("Started at ", start_time)

Started at  2021-02-06 22:19:19.816540


In [3]:
# Starting H2O

import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

3.32.0.3
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.1+13-LTS, mixed mode)
  Starting server from C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\USER-PC\AppData\Local\Temp\tmpxxj35foc
  JVM stdout: C:\Users\USER-PC\AppData\Local\Temp\tmpxxj35foc\h2o_USER_PC_started_from_python.out
  JVM stderr: C:\Users\USER-PC\AppData\Local\Temp\tmpxxj35foc\h2o_USER_PC_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Helsinki
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.3
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_USER_PC_7laaw6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,16 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
in_kaggle = False


def get_data_file_path(is_in_kaggle: bool) -> Tuple[str, str, str]:
    train_path = ''
    test_path = ''
    sample_submission_path = ''

    if is_in_kaggle:
        # running in Kaggle, inside the competition
        train_path = '../input/tabular-playground-series-feb-2021/train.csv'
        test_path = '../input/tabular-playground-series-feb-2021/test.csv'
        sample_submission_path = '../input/tabular-playground-series-feb-2021/sample_submission.csv'
    else:
        # running locally
        train_path = 'data/train.csv'
        test_path = 'data/test.csv'
        sample_submission_path = 'data/sample_submission.csv'

    return train_path, test_path, sample_submission_path

In [5]:
%%time
# get the training set and labels
train_set_path, test_set_path, sample_subm_path = get_data_file_path(in_kaggle)

df_train = pd.read_csv(train_set_path)
df_test = pd.read_csv(test_set_path)

subm = pd.read_csv(sample_subm_path)

# list of basic raw features
feature_list = [col for col in df_train.columns if col.startswith('cont')]

Wall time: 1.8 s


In [6]:
# drop sig_id from train and test sets
df_train = df_train.drop(['id'], axis=1, errors='ignore')
df_test = df_test.drop(['id'], axis=1, errors='ignore')

In [7]:
%%time
train = h2o.H2OFrame(df_train)
test = h2o.H2OFrame(df_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Wall time: 19.6 s


In [8]:
%%time
x = test.columns
y = 'target'

Wall time: 0 ns


In [9]:
# Run AutoML for up to 2000 models to generate, up to 12 h in terms of duration (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=2000, seed=47, max_runtime_secs=43200)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
22:20:07.157: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [10]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=200)  # Print top 200 rows instead of default (10 rows); rows=lb.nrows would output the entire leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_grid__1_AutoML_20210206_222007_model_36,0.715345,0.845781,0.715345,0.706953,0.102548
GBM_grid__1_AutoML_20210206_222007_model_32,0.715475,0.845858,0.715475,0.706696,0.102526
GBM_grid__1_AutoML_20210206_222007_model_5,0.716052,0.846198,0.716052,0.707152,0.102593
GBM_1_AutoML_20210206_222007,0.71624,0.84631,0.71624,0.707167,0.102601
GBM_grid__1_AutoML_20210206_222007_model_58,0.716498,0.846462,0.716498,0.707412,0.102627
GBM_2_AutoML_20210206_222007,0.71679,0.846635,0.71679,0.707279,0.102634
GBM_grid__1_AutoML_20210206_222007_model_39,0.717086,0.84681,0.717086,0.707795,0.102679
GBM_grid__1_AutoML_20210206_222007_model_46,0.717183,0.846866,0.717183,0.707678,0.10268
GBM_3_AutoML_20210206_222007,0.717205,0.84688,0.717205,0.707294,0.102659
GBM_grid__1_AutoML_20210206_222007_model_25,0.717365,0.846974,0.717365,0.707284,0.102669


In [11]:
# The leader model is stored here
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid__1_AutoML_20210206_222007_model_36


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,119.0,119.0,87609.0,6.0,6.0,6.0,26.0,64.0,53.941177




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.6999385954092944
RMSE: 0.8366233294675056
MAE: 0.6993685530662537
RMSLE: 0.10146690573638509
Mean Residual Deviance: 0.6999385954092944

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 0.7153452455808479
RMSE: 0.8457808496181785
MAE: 0.7069531475811812
RMSLE: 0.10254807703342084
Mean Residual Deviance: 0.7153452455808479

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.70695317,0.0029271368,0.7070669,0.7073414,0.70585734,0.71129555,0.7032046
1,mean_residual_deviance,0.71534526,0.0045977687,0.71441305,0.7149606,0.7161012,0.72206044,0.70919096
2,mse,0.71534526,0.0045977687,0.71441305,0.7149606,0.7161012,0.72206044,0.70919096
3,r2,0.09137465,0.0015083019,0.09099272,0.08986585,0.09011932,0.092671275,0.09322409
4,residual_deviance,0.71534526,0.0045977687,0.71441305,0.7149606,0.7161012,0.72206044,0.70919096
5,rmse,0.84577733,0.002717314,0.84522957,0.8455534,0.84622765,0.8497414,0.8421348
6,rmsle,0.10254739,4.1926216E-4,0.10240813,0.10247808,0.10294894,0.10295044,0.101951376



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2021-02-06 23:10:25,32 min 50.476 sec,0.0,0.887293,0.741991,0.787289
1,,2021-02-06 23:10:26,32 min 50.811 sec,5.0,0.875962,0.733694,0.767309
2,,2021-02-06 23:10:26,32 min 51.107 sec,10.0,0.869270,0.728159,0.755630
3,,2021-02-06 23:10:26,32 min 51.408 sec,15.0,0.863162,0.722944,0.745049
4,,2021-02-06 23:10:27,32 min 51.709 sec,20.0,0.859002,0.719260,0.737884
5,,2021-02-06 23:10:27,32 min 52.015 sec,25.0,0.856057,0.716799,0.732833
6,,2021-02-06 23:10:27,32 min 52.309 sec,30.0,0.853585,0.714611,0.728607
7,,2021-02-06 23:10:28,32 min 52.591 sec,35.0,0.851835,0.713025,0.725623
8,,2021-02-06 23:10:28,32 min 52.885 sec,40.0,0.850129,0.711445,0.722719
9,,2021-02-06 23:10:28,32 min 53.153 sec,45.0,0.848695,0.710197,0.720283



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,cat1,15118.857422,1.000000,0.109619
1,cont8,12836.661133,0.849050,0.093072
2,cont0,12339.690430,0.816179,0.089469
3,cont13,10550.382812,0.697829,0.076495
4,cat9,10356.431641,0.685001,0.075089
5,cat2,9204.409180,0.608803,0.066736
6,cont11,8824.727539,0.583690,0.063984
7,cont9,8015.886719,0.530191,0.058119
8,cont5,6701.562988,0.443259,0.048590
9,cont6,6021.437988,0.398273,0.043658



See the whole table with table.as_data_frame()


In [12]:
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

preds = aml.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [13]:
preds.as_data_frame().values.flatten()

array([7.64873834, 7.8351921 , 7.6319819 , ..., 7.47117878, 7.45805109,
       7.35028207])

In [14]:
# Public LB 0.84611
subm['target'] = preds.as_data_frame().values.flatten()
subm.to_csv('h2o_automl_baseline_submission.csv', index=False)

In [15]:
print('We are done. That is all, folks!')
finish_time = dt.datetime.now()
print("Finished at ", finish_time)
elapsed = finish_time - start_time
print("Elapsed time: ", elapsed)

We are done. That is all, folks!
Finished at  2021-02-07 06:33:06.099998
Elapsed time:  8:13:46.283458
